In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import re
import math
import numpy as np
import time

In [2]:
startnum = 1

In [3]:
# アクセスURL
url = 'http://fishpix.kahaku.go.jp/fishimage/search?START=%d&JPN_FAMILY=&FAMILY=Chaetodontidae&JPN_NAME=&SPECIES=&LOCALITY=&FISH_Y=&FISH_M=&FISH_D=&PERSON=&PHOTO_ID=&JPN_FAMILY_OPT=1&FAMILY_OPT=0&JPN_NAME_OPT=1&SPECIES_OPT=1&LOCALITY_OPT=1&PERSON_OPT=1&PHOTO_ID_OPT=1'%startnum

In [4]:
def url_generate(url):
#     if startnum == 1:
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    text = soup.find('span', class_= 'captionTimes').text
    num = text.split("件")[0].split()[0]
    print('%s images were found'%num)
    page = math.ceil(int(num)/20.)
    page_nums = list(np.arange(1, page*20, 20))
    return page_nums

In [5]:
page_nums = url_generate(url)

2832 images were found


In [6]:
def information_getter(url):
    '''
    urlを渡すと情報を返す
    '''
    
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    result = soup.findAll('span', class_= 'result')
    names = []
    for x in result:
        names.append(x.text)
    result_ = soup.findAll('span', class_= 'resultHelvetica')
    namesHel = []
    for x in result_:
        namesHel.append(x.text)
    
    caption = soup.findAll('span', class_= 'captionTimes')
    caps = []
    for x in caption:
        cap = x.find('a',href=re.compile('^detail'))
        if cap != None:
            caps.append(cap.text)
    
    imgsoup = soup.findAll("img")
    img_path = []
    for x in imgsoup:
        if x['height'] == "130":
            img_path.append(x['src'])

    return names, namesHel, caps, img_path

In [7]:
def dataframe_maker(page_nums):
    df = pd.DataFrame(columns = ['name', 'nameEng', 'caption', 'URL'])
    
    for n in page_nums:
        url = 'http://fishpix.kahaku.go.jp/fishimage/search?START=%d&JPN_FAMILY=&FAMILY=Chaetodontidae&JPN_NAME=&SPECIES=&LOCALITY=&FISH_Y=&FISH_M=&FISH_D=&PERSON=&PHOTO_ID=&JPN_FAMILY_OPT=1&FAMILY_OPT=0&JPN_NAME_OPT=1&SPECIES_OPT=1&LOCALITY_OPT=1&PERSON_OPT=1&PHOTO_ID_OPT=1'%n
        names, namesHel, caps, img_url = information_getter(url)
        df_batch = pd.DataFrame([names, namesHel, caps, img_url],\
                                index=['name', 'nameEng', 'caption', 'URL']).transpose()
#         display(df_batch.head())
        df = pd.concat([df, df_batch])
#         display(df.tail())
        print(n)
#         display(names)
#         display(caps)
        time.sleep(2)
    return df

In [9]:
df = dataframe_maker(page_nums)

1
21
41
61
81
101
121
141
161
181
201
221
241
261
281
301
321
341
361
381
401
421
441
461
481
501
521
541
561
581
601
621
641
661
681
701
721
741
761
781
801
821
841
861
881
901
921
941
961
981
1001
1021
1041
1061
1081
1101
1121
1141
1161
1181
1201
1221
1241
1261
1281
1301
1321
1341
1361
1381
1401
1421
1441
1461
1481
1501
1521
1541
1561
1581
1601
1621
1641
1661
1681
1701
1721
1741
1761
1781
1801
1821
1841
1861
1881
1901
1921
1941
1961
1981
2001
2021
2041
2061
2081
2101
2121
2141
2161
2181
2201
2221
2241
2261
2281
2301
2321
2341
2361
2381
2401
2421
2441
2461
2481
2501
2521
2541
2561
2581
2601
2621
2641
2661
2681
2701
2721
2741
2761
2781
2801
2821


In [10]:
df.count()

name       2832
nameEng    2832
caption    2832
URL        2832
dtype: int64

In [13]:
df['name'].value_counts()

トゲチョウチョウウオ                            159
ムレハタタテダイ                              140
チョウチョウウオ                              136
フウライチョウチョウウオ                          122
チョウハン                                 106
ミゾレチョウチョウウオ                            99
シラコダイ                                  93
アケボノチョウチョウウオ                           88
フエヤッコダイ                                86
ハタタテダイ                                 85
ミナミハタタテダイ                              78
ミスジチョウチョウウオ                            74
タキゲンロクダイ                               68
ヤリカタギ                                  68
ユウゼン                                   59
セグロチョウチョウウオ                            56
シチセンチョウチョウウオ                           55
ゴマチョウチョウウオ                             53
ゲンロクダイ                                 53
ウミヅキチョウチョウウオ                           52
イッテンチョウチョウウオ                           50
カガミチョウチョウウオ                            50
ハナグロチョウチョウウオ                           48
スミツキトノサマダイ                        

In [14]:
df.to_csv("butterfly.csv")

In [20]:
names, namesHel, caps, img_url = information_getter(url)
df_batch = pd.DataFrame([names, namesHel, caps, img_url], index=['name', 'nameEng', 'caption', 'URL']).transpose()

In [22]:
df = pd.concat([df, df_batch])
df

,name,nameEng,caption,URL
0,トゲチョウチョウウオ,"Chaetodon auriga Forsskal, 1775",KPM-NR 10582,../photos/NR0010/10582AI.jpg
1,スミツキトノサマダイ,"Chaetodon plebeius Cuvier, 1831",KPM-NR 10583,../photos/NR0010/10583AI.jpg
2,ツキチョウチョウウオ,"Chaetodon wiebeli Kaup, 1863",KPM-NR 10584,../photos/NR0010/10584AI.jpg
3,ツキチョウチョウウオ,"Chaetodon wiebeli Kaup, 1863",KPM-NR 10585,../photos/NR0010/10585AI.jpg
4,チョウチョウウオ,"Chaetodon auripes Jordan et Snyder, 1901",KPM-NR 10586,../photos/NR0010/10586AI.jpg
5,ミゾレチョウチョウウオ,"Chaetodon kleinii Bloch, 1790",KPM-NR 10587,../photos/NR0010/10587AI.jpg
6,ミナミハタタテダイ,"Heniochus chrysostomus Cuvier, 1831",KPM-NR 10592,../photos/NR0010/10592AI.jpg
7,ムレハタタテダイ,"Heniochus diphreutes Jordan, 1903",KPM-NR 10593,../photos/NR0010/10593AI.jpg
8,ツーアイド・コーラルフィッシュ,"Coradion melanopus (Cuvier, 1831)",KPM-NR 10621,../photos/NR0010/10621AI.jpg
9,ミカドチョウチョウウオ,"Chaetodon baronessa Cuvier, 1829",KPM-NR 10624,../photos/NR0010/10624AI.jpg


In [10]:
url = 'http://fishpix.kahaku.go.jp/fishimage/search?START=2781&JPN_FAMILY=&FAMILY=Chaetodontidae&JPN_NAME=&SPECIES=&LOCALITY=&FISH_Y=&FISH_M=&FISH_D=&PERSON=&PHOTO_ID=&JPN_FAMILY_OPT=1&FAMILY_OPT=0&JPN_NAME_OPT=1&SPECIES_OPT=1&LOCALITY_OPT=1&PERSON_OPT=1&PHOTO_ID_OPT=1'

In [11]:
response = urllib.request.urlopen(url)

In [12]:
soup = BeautifulSoup(response, "html.parser")

In [15]:
result = soup.findAll('span', class_= 'result')
names = []
for x in result:
    names.append(x.text)
print(names)
len(names)

['レモンチョウチョウウオ', 'オウギチョウチョウウオ', 'ゴマチョウチョウウオ', 'ミナミハタタテダイ', 'ゴマチョウチョウウオ', 'ゴマチョウチョウウオ', 'ドットアンドダッシュ・バタフライフィッシュ', 'ベニオチョウチョウウオ', 'ムレハタタテダイ', 'シラコダイ', 'ゲンロクダイ', 'ヤリカタギ', 'ヤリカタギ', 'ヤリカタギ', 'アケボノチョウチョウウオ', 'ハシナガチョウチョウウオ', 'ハシナガチョウチョウウオ', 'ヤスジチョウチョウウオ', 'イッテンチョウチョウウオ', 'ドットアンドダッシュ・バタフライフィッシュ']


20

In [14]:
result_ = soup.findAll('span', class_= 'resultHelvetica')
namesHel = []
for x in result_:
    namesHel.append(x.text)
print(namesHel)

['Chaetodon semeion Bleeker, 1855', 'Chaetodon meyeri Bloch et Schneider, 1801', 'Chaetodon citrinellus Cuvier, 1831', 'Heniochus chrysostomus Cuvier, 1831', 'Chaetodon citrinellus Cuvier, 1831', 'Chaetodon citrinellus Cuvier, 1831', 'Chaetodon pelewensis Kner, 1868', 'Chaetodon mertensii Cuvier, 1831', 'Heniochus diphreutes Jordan, 1903', 'Chaetodon nippon Doderlein, 1883', 'Chaetodon modestus Temminck et Schlegel, 1844', 'Chaetodon trifascialis Quoy et Gaimard, 1825', 'Chaetodon trifascialis Quoy et Gaimard, 1825', 'Chaetodon trifascialis Quoy et Gaimard, 1825', 'Chaetodon melannotus Bloch et Schneider, 1801', 'Chelmon rostratus (Linnaeus, 1758)', 'Chelmon rostratus (Linnaeus, 1758)', 'Chaetodon octofasciatus Bloch, 1787', 'Chaetodon unimaculatus Bloch, 1787', 'Chaetodon pelewensis Kner, 1868']


In [10]:
soup.findAll('span', class_= 'captionTimes')

[<span class="captionTimes"><font color="#1235ff">
       2832件中 1-20件目</font>
 </span>,
 <span class="captionTimes"><font color="#1235ff"><a href="search?START=1&amp;JPN_FAMILY=&amp;FAMILY=Chaetodontidae&amp;JPN_NAME=&amp;SPECIES=&amp;LOCALITY=&amp;FISH_Y=&amp;FISH_M=&amp;FISH_D=&amp;PERSON=&amp;PHOTO_ID=&amp;JPN_FAMILY_OPT=1&amp;FAMILY_OPT=0&amp;JPN_NAME_OPT=1&amp;SPECIES_OPT=1&amp;LOCALITY_OPT=1&amp;PERSON_OPT=1&amp;PHOTO_ID_OPT=1">[1-20]</a></font></span>,
 <span class="captionTimes"><font color="#1235ff"><a href="search?START=21&amp;JPN_FAMILY=&amp;FAMILY=Chaetodontidae&amp;JPN_NAME=&amp;SPECIES=&amp;LOCALITY=&amp;FISH_Y=&amp;FISH_M=&amp;FISH_D=&amp;PERSON=&amp;PHOTO_ID=&amp;JPN_FAMILY_OPT=1&amp;FAMILY_OPT=0&amp;JPN_NAME_OPT=1&amp;SPECIES_OPT=1&amp;LOCALITY_OPT=1&amp;PERSON_OPT=1&amp;PHOTO_ID_OPT=1">[21-40]</a></font></span>,
 <span class="captionTimes"><font color="#1235ff"><a href="search?START=41&amp;JPN_FAMILY=&amp;FAMILY=Chaetodontidae&amp;JPN_NAME=&amp;SPECIES=&amp;LOCALITY=

In [17]:
caption = soup.findAll('span', class_= 'captionTimes')
display(caption)
caps = []
for x in caption:
    cap = x.find('a',href=re.compile('^detail'))
    if cap != None:
        caps.append(cap.text)
print(caps)

[<span class="captionTimes"><font color="#1235ff">
       2832件中 2781-2800件目</font>
 </span>,
 <span class="captionTimes"><font color="#1235ff"><a href="search?START=2701&amp;JPN_FAMILY=&amp;FAMILY=Chaetodontidae&amp;JPN_NAME=&amp;SPECIES=&amp;LOCALITY=&amp;FISH_Y=&amp;FISH_M=&amp;FISH_D=&amp;PERSON=&amp;PHOTO_ID=&amp;JPN_FAMILY_OPT=1&amp;FAMILY_OPT=0&amp;JPN_NAME_OPT=1&amp;SPECIES_OPT=1&amp;LOCALITY_OPT=1&amp;PERSON_OPT=1&amp;PHOTO_ID_OPT=1">[2701-2720]</a></font></span>,
 <span class="captionTimes"><font color="#1235ff"><a href="search?START=2721&amp;JPN_FAMILY=&amp;FAMILY=Chaetodontidae&amp;JPN_NAME=&amp;SPECIES=&amp;LOCALITY=&amp;FISH_Y=&amp;FISH_M=&amp;FISH_D=&amp;PERSON=&amp;PHOTO_ID=&amp;JPN_FAMILY_OPT=1&amp;FAMILY_OPT=0&amp;JPN_NAME_OPT=1&amp;SPECIES_OPT=1&amp;LOCALITY_OPT=1&amp;PERSON_OPT=1&amp;PHOTO_ID_OPT=1">[2721-2740]</a></font></span>,
 <span class="captionTimes"><font color="#1235ff"><a href="search?START=2741&amp;JPN_FAMILY=&amp;FAMILY=Chaetodontidae&amp;JPN_NAME=&amp;S

['KPM-NR 9771', 'KPM-NR 9772', 'KPM-NR 97778', 'KPM-NR 97783', 'KPM-NR 97792', 'KPM-NR 97797', 'KPM-NR 97801', 'KPM-NR 97855', 'KPM-NR 97984', 'KPM-NR 98005', 'KPM-NR 981', 'KPM-NR 98147', 'KPM-NR 98152', 'KPM-NR 98208', 'KPM-NR 9825', 'KPM-NR 98389', 'KPM-NR 98400', 'KPM-NR 98456', 'KPM-NR 98945', 'KPM-NR 98964']


In [98]:
text = soup.find('span', class_= 'captionTimes').text
num = text.split("件")[0].split()[0]
print(num)

2832


In [22]:
imgsoup = soup.findAll("img")
display(imgsoup)

[<img align="absmiddle" alt="前へ" border="0" height="13" src="images/L.jpg" width="13"/>,
 <img align="absmiddle" alt="次へ" border="0" height="13" src="images/R.jpg" width="13"/>,
 <img border="0" height="130" src="../photos/NR0009/9771AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0009/9772AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0097/97778AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0097/97783AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0097/97792AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0097/97797AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0097/97801AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0097/97855AI.jpg" vspace="2" width="130"/>,
 <img border="0" height="130" src="../photos/NR0097/97984AI.jpg" vspace="2" width="130"/>,
 <img

In [26]:
img_path = []

for x in imgsoup:
    if x['height'] == "130":
        img_path.append(x['src'])

In [27]:
print(len(img_path))
print(img_path)

20
['../photos/NR0009/9771AI.jpg', '../photos/NR0009/9772AI.jpg', '../photos/NR0097/97778AI.jpg', '../photos/NR0097/97783AI.jpg', '../photos/NR0097/97792AI.jpg', '../photos/NR0097/97797AI.jpg', '../photos/NR0097/97801AI.jpg', '../photos/NR0097/97855AI.jpg', '../photos/NR0097/97984AI.jpg', '../photos/NR0098/98005AI.jpg', '../photos/NR0000/981AI.jpg', '../photos/NR0098/98147AI.jpg', '../photos/NR0098/98152AI.jpg', '../photos/NR0098/98208AI.jpg', '../photos/NR0009/9825AI.jpg', '../photos/NR0098/98389AI.jpg', '../photos/NR0098/98400AI.jpg', '../photos/NR0098/98456AI.jpg', '../photos/NR0098/98945AI.jpg', '../photos/NR0098/98964AI.jpg']
